Lista 3

In [1]:
import numpy as np
from scipy.linalg import solve

In [187]:
A = np.array([
    [1, 1/2, 1/3, 1/4, 1/5],
    [1/2, 1/3, 1/4, 1/5, 1/6],
    [1/3, 1/4, 1/5, 1/6, 1/7],
    [1/4, 1/5, 1/6, 1/7, 1/8],
    [1/5, 1/6, 1/7, 1/8, 1/9]
])

b = np.array([5,4,3,2,1])

augmented_matrix = np.column_stack((A, b))

In [188]:
def lu(mat):
    mat = mat.astype(float)
    n = len(mat)
    L, U = np.zeros((n,n)), np.zeros((n,n))

    for i in range(n):
        for k in range(i, n):
            sum_ = 0
            for j in range(i):
                sum_ += (L[i][j] * U[j][k])
            U[i][k] = mat[i][k] - sum_

        for k in range(i, n):
            if (i == k):
                L[i][i] = 1 
            else:
                sum_ = 0
                for j in range(i):
                    sum_ += (L[k][j] * U[j][i])
                L[k][i] = (mat[k][i] - sum_) / U[i][i]
    return L, U


In [189]:
L,U = lu(A)

In [190]:
y = solve(L, b).astype(float)

In [191]:
x = solve(U, y).astype(float)

In [192]:
print(x)

[   -95.           2159.99999999 -10709.99999998  17919.99999997
  -9449.99999999]


In [193]:
Ax = np.dot(A, x).astype(float)

In [194]:
r = b.astype(float) - Ax.astype(float)

In [195]:
print(r)

[ 1.42108547e-14 -3.90798505e-13 -3.34843264e-13  4.54747351e-13
  1.94400052e-13]


In [196]:
norma_r = np.linalg.norm(r,np.inf).astype(float)
norma_Ax = np.linalg.norm(Ax, np.inf).astype(float)

In [197]:
print(norma_r, norma_Ax)

4.547473508864641e-13 4.999999999999986


In [198]:
def dokladnosc_maszynowa(x_0):
    x = x_0
    while x + 1 != 1:
        x /= 2
    return x

In [199]:
u = dokladnosc_maszynowa(1) * 10**2

In [200]:
print(u)

1.1102230246251565e-14


In [201]:
delta_x = solve(A, r)

In [274]:
def iteracyjne_poprawianie_rozwiazan(A, b):
    L,U = lu(A)#rozkład LU
    y = solve(L, b)#rozwiązujemy za pomocą LU
    x = [1,1,1,1,1]
    x_0 = x
    Ax = np.dot(A, x)
    r = (b.astype(np.float64) - Ax.astype(np.float64))#reszty
    u = dokladnosc_maszynowa(1) * 10**(-10)
    while np.linalg.norm(r,np.inf) > np.linalg.norm(Ax, np.inf) * u:
        delta_x = solve(A, r)
        x += (delta_x * x)
        r = b.astype(np.float64) - np.dot(A, x).astype(np.float64)
    return x, x_0

In [275]:
print(iteracyjne_poprawianie_rozwiazan(A, b))

C:\Users\rafal\AppData\Local\Temp\ipykernel_36612\2916120735.py:11: RuntimeWarning: overflow encountered in multiply
  x += (delta_x * x)


ValueError: array must not contain infs or NaNs

In [182]:
import numpy as np
from scipy.linalg import lu, solve

def iteracyjne_poprawianie_rozwiazan(A, b):
    # Work with single precision for the initial matrix and vector
    A = A.astype(np.float32)
    b = b.astype(np.float32)
    
    # Perform LU decomposition in double precision
    A_double = A.astype(np.float64)
    L, U = lu(A_double, permute_l=True)  # LU in double precision
    
    # Initial solution in single precision
    y = solve(L, b.astype(np.float64))
    x = solve(U, y).astype(np.float32)
    
    # Machine epsilon for single precision
    u = np.finfo(np.float32).eps

    # Iterative refinement
    while True:
        # Compute the residual in double precision
        r = (b - A @ x).astype(np.float64)
        
        # Check if the residual is within the tolerance
        if np.linalg.norm(r, np.inf) <= np.linalg.norm(A, np.inf) * u:
            break
        
        # Solve for correction term in double precision
        delta_x = solve(A_double, r).astype(np.float32)
        
        # Update x in single precision
        x += delta_x
    
    return x

# Define the Hilbert matrix and vector b in single precision
A = np.array([
    [1, 1/2, 1/3, 1/4, 1/5],
    [1/2, 1/3, 1/4, 1/5, 1/6],
    [1/3, 1/4, 1/5, 1/6, 1/7],
    [1/4, 1/5, 1/6, 1/7, 1/8],
    [1/5, 1/6, 1/7, 1/8, 1/9]
], dtype=np.float32)

b = np.array([5, 4, 3, 2, 1], dtype=np.float32)

# Call the iterative refinement function
solution = iteracyjne_poprawianie_rozwiazan(A, b)
print("Refined solution:", solution)


Refined solution: [   -95.14292   2162.6323  -10721.137    17936.541    -9457.98   ]
